<b><span style="color:#E04728;font-size:26pt">Die Periheldrehung des Merkurs</span></b>

Dieses Notebook simuliert die Bewegung des Merkurs, optional unter Berücksichtigung der Relativitätstheorie.

# Einrichtung der Simulation

Lade alle nötigen Funktionen und richte Grafikausgabe ein.

In [ ]:
from graphics import *

Definiere physikalische Parameter. Die folgenden Parameter wurden berechnet mittels https://nssdc.gsfc.nasa.gov/planetary/factsheet

In [ ]:
rM0 = 4.60     # Startradius des Merkurorbits
vM0 = 5.10e-1  # Startgeschwindigkeit des Merkurs
c_a = 9.90e-1  # Basisbeschleunigung des Merkurs
rS  = 2.95e-7  # Schwarzschildradius der Sonne
rL2 = 8.19e-7  # Spezifischer Drehimpuls

Für die physikalischen Berechnungen und grafische Ausgabe brauchen wir Vektoren. Die Startposition und -geschwindigkeit des Merkurs können wie folgt definiert werden.

In [ ]:
vec_rM0 = vector(0, rM0, 0) # Startposition des Merkurs
vec_vM0 = vector(vM0, 0, 0) # Startgeschwindigkeit des Merkurs

Definiere Simulationsparameter. Diese werden nur von der numerischen Simulation gebraucht und haben keine physikalische Bedeutung.

In [ ]:
dt = 2. * vM0 / c_a / 20  # Zeitschritt
max_time = 10000*dt       # Maximale Simulationszeit

# Der Merkurorbit mit Klassischer Gravitation

In diesem Abschnitt wird der Merkurorbit mittels klassischer (Newtonscher) Mechanik simuliert.<br>
Definiere dazu eine Funktion, die Ort und Geschwindigkeit des Merkurs in der Zeit entwickelt. Dazu wird die Gravitationskraft der Sonne auf den Merkur mit klassischer (Newtonscher) Mechanik berechnet. Anschließend wird der Merkur ein kurzes Stück geradlinig bewegt.

In [ ]:
def evolve_mercury_classic(vec_rM_old, vec_vM_old):
    """
    Berechne einen Zeitschritt des Merkurs.
    Argumente:
         - vec_rM_old: alter Positionsvektor des Merkur
         - vec_vM_old: alter Geschwindigkeitsvektor des Merkur
    Rückgabe:
         - vec_rM_new: neuer Positionsvektor des Merkur
         - vec_vM_new: neuer Geschwindigkeitsvektor des Merkur
    """

    # Berechne die absolute Beschleunigung
    aMS = c_a / vec_rM_old.mag**2
    # Multipliziere mit der Richtung um den Beschleunigungsvektor zu erhalten
    vec_aMS = - aMS * (vec_rM_old / vec_rM_old.mag)
    # Aktualisiere Geschwindigkeitsvektor
    vec_vM_new = vec_vM_old + vec_aMS * dt
    # Aktualisiere Positionsvektor
    vec_rM_new = vec_rM_old + vec_vM_new * dt
    
    return vec_rM_new, vec_vM_new

In [ ]:
# Erzeuge eine neue Grafikausgabe weiter unten
canvas()

# Zeichne Kugeln and den Startpositionen; M = Merkur, S = Sonne
M = sphere(pos=vec_rM0, radius=0.5, color=color.red)
S = sphere(pos=vector(0, 0, 0), radius=1.5, color=color.yellow)
# Setze Startgeschwindigkeiten
M.velocity = vec_vM0
S.velocity = vector(0, 0, 0)

# Mache die Flugbahn des Merkurs sichtbar
M.trajectory = curve(color=color.black, radius=0.02)

# Führe die Simulation for die gegebene Zeit aus und zeichne die Trajektorie
time = 0  # Aktuelle Simulationszeit
while time < max_time:
    # Setzte Bildrate
    rate(100)
    # Füge die aktuelle Position zur Trajektorie hinzu
    M.trajectory.append(pos=M.pos)
    # Aktualisiere Position und Geschwindigkeit
    M.pos, M.velocity = evolve_mercury_classic(M.pos, M.velocity)
    # Setze Simulationszeit weiter
    time = time + dt

# Der Merkurorbit mit Relativistischer Gravitation

In diesem Abschnitt wird schließlich der Effekt der Allgemeinen Relativitätstheorie auf den Merkurorbit simuliert.<br>
Die unten definierte Funktion ist die gleiche wie zuvor aber mit einem extra Schritt, der den relativistischen Anteil berechnet.

In [ ]:
def evolve_mercury_relativistic(vec_rM_old, vec_vM_old, a, b):
    """
    Berechne einen Zeitschritt des Merkurs.
    Argumente:
         - vec_rM_old: alter Positionsvektor des Merkur
         - vec_vM_old: alter Geschwindigkeitsvektor des Merkur
         - a: Stärke des 1/r**3 Terms in der Kraft
         - b: Stärke des 1/r**4 Terms in der Kraft
    Rückgabe:
         - vec_rM_new: neuer Positionsvektor des Merkur
         - vec_vM_new: neuer Geschwindigkeitsvektor des Merkur
    """

    # Berechne den Faktor aus der Allgemeinen Relativitätstheorie
    fact = 1 + a * rS / vec_rM_old.mag + b * rL2 / vec_rM_old.mag**2
    
    # Berechne die absolute Beschleunigung
    aMS = c_a * fact / vec_rM_old.mag**2
    # Multipliziere mit der Richtung um den Beschleunigungsvektor zu erhalten
    vec_aMS = - aMS * (vec_rM_old / vec_rM_old.mag)
    # Aktualisiere Geschwindigkeitsvektor
    vec_vM_new = vec_vM_old + vec_aMS * dt
    # Aktualisiere Positionsvektor
    vec_rM_new = vec_rM_old + vec_vM_new * dt
    
    return vec_rM_new, vec_vM_new

Die folgenden Parameter bestimmen die Stärke der relativistischen Kraft, siehe Poster.
Hier müssen sehr große Werte gewählt werden, damit in der Simulation etwas zu sehen ist.

In [ ]:
a = 10**6  # Stärke des 1/r**3 tTerms
b = 0      # Stärke des 1/r**4 Terms

In [ ]:
# Erzeuge eine neue Grafikausgabe weiter unten
canvas()

# Zeichne Kugeln and den Startpositionen; M = Merkur, S = Sonne
M = sphere(pos=vec_rM0, radius=0.5, color=color.red)
S = sphere(pos=vector(0, 0, 0), radius=1.5, color=color.yellow)
# Setze Startgeschwindigkeiten
M.velocity = vec_vM0
S.velocity = vector(0, 0, 0)

# Mache die Flugbahn des Merkurs sichtbar
M.trajectory = curve(color=color.black, radius=0.02)

# Führe die Simulation for die gegebene Zeit aus und zeichne die Trajektorie
time = 0  # Aktuelle Simulationszeit
while time < max_time:
    # Setzte Bildrate
    rate(100)
    # Füge die aktuelle Position zur Trajektorie hinzu
    M.trajectory.append(pos=M.pos)
    # Aktualisiere Position und Geschwindigkeit
    M.pos, M.velocity = evolve_mercury_relativistic(M.pos, M.velocity, a, b)
    # Setze Simulationszeit weiter
    time = time + dt